In [ ]:
import pandas as pd
import nltk
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn import model_selection, svm
from sklearn.metrics import accuracy_score
from sklearn.neighbors import KNeighborsClassifier

stopwords = nltk.corpus.stopwords.words('english')
lemmetizer = nltk.WordNetLemmatizer()
vectorizer = CountVectorizer()

def remove_stop_words(text):
    text = [w for w in text if w.lower() not in stopwords]
    return text

def lemmetize_words(word_list):
    lemmetized = [lemmetizer.lemmatize(w) for w in word_list]
    return lemmetized

def fix_text(row):
    return ','.join(row)

df = pd.read_csv('train.csv')
df = df.dropna()
df['text'] = df['text'].apply(nltk.word_tokenize)
df['text'] = df['text'].apply(remove_stop_words)
df['text'] = df['text'].apply(lemmetize_words)
df['text'] = df['text'].apply(fix_text)

train_x, test_x, train_y, test_y = model_selection.train_test_split(df['text'], df['label'], test_size=0.2)

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(df['text'])
Train_X_Tfidf = Tfidf_vect.transform(train_x)
Test_X_Tfidf = Tfidf_vect.transform(test_x)

In [ ]:
SVM = svm.SVC()
SVM.fit(Train_X_Tfidf,train_y)
predictions_SVM = SVM.predict(Test_X_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, test_y)*100)

In [ ]:
from sklearn.model_selection import cross_val_score, GridSearchCV
knn = KNeighborsClassifier()
parameters = {'n_neighbors' : [1,10]}
grid = GridSearchCV(knn, parameters)
grid.fit(Train_X_Tfidf, train_y)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(Train_X_Tfidf, train_y)
knn_predict = knn.predict(Test_X_Tfidf)
print(accuracy_score(knn_predict, test_y))